<a href="https://colab.research.google.com/github/Banafshehkh/C.elegans-Control-Backbone/blob/main/Destroying_the_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this level we systematically destroy the network and see if it can reproduce the behavior

# **Initialization:** A population of potential solutions is randomly generated. Each solution is represented as a set of parameters or a string of values known as a chromosome.